# WORD EMBEDDING

## 参考资料

[什么是 word embedding?](https://www.zhihu.com/question/32275069/answer/61059440)

[WORD EMBEDDINGS: ENCODING LEXICAL SEMANTICS](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#sphx-glr-beginner-nlp-word-embeddings-tutorial-py)

[YJango的Word Embedding--介绍](https://zhuanlan.zhihu.com/p/27830489)

## 概念解析

## 代码解析

与我们制作独热向量（one-hot vectors）时候为每个单词定义一个唯一索引类似，在使用词嵌入的时候我们也需要为每个单词定义索引，方便查找。

词嵌入被储存为了一个 $ |V|×D $ 的矩阵，其中 $ D $ 是嵌入的维度，所以某单词的索引为 $ i $，那么它就被嵌入在矩阵的第 $ i $ 行中。在代码中，从单词到索引的映射是一个名叫 word_to_ix 的字典。

在 PyTorch 中，torch.nn.Embedding 可以完成词嵌入功能，包含两个参数：词表大小和嵌入维度。

使用 torch.LongTensor 进行索引（这是一个 64-bit integer，带符号数）。

In [1]:
# Author: Robert Guthrie
# 补充：Dr_David_S

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)  # 设定随机种子，返回一个torch.Generator对象

现在我们设置一个字典，用最简单的 hello world 做例子：

In [2]:
word_to_ix = {"hello": 0, "world": 1}
word_to_ix

{'hello': 0, 'world': 1}

使用 torch.nn.Embedding 方法准备进行词嵌入，参数的意义是：有2个词语，5个维度。

In [3]:
embeds = nn.Embedding(num_embeddings=2, embedding_dim=5)  # 2 words in vocab, 5 dimensional embeddings
embeds

Embedding(2, 5)

现在将 hello 对应的值 0 转换为一个 longtensor 类型,两种方法都行。

In [4]:
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
lookup_tensor

tensor([0])

In [5]:
lookup_tensor_test = torch.LongTensor([word_to_ix["hello"]])
lookup_tensor_test

tensor([0])

现在使用之前定义的 embeds 模型对 hello 这个次进行词嵌入（或者说转换为词向量）：

In [6]:
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


原来的代码到此就结束了，但是这里我们希望再试试对 world 的词嵌入：

In [7]:
lookup_tensor2 = torch.tensor([word_to_ix["world"]], dtype=torch.long)
lookup_tensor2

tensor([1])

In [8]:
world_embed = embeds(lookup_tensor2)
print(world_embed)

tensor([[-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]],
       grad_fn=<EmbeddingBackward>)


从上述结果来看，似乎对于 hello 的 embedding 没有涉及到 world？

事实上，embeds只是一个随机生成的矩阵，目前其中的向量并不能代表任何意义，如下：

In [9]:
embeds.weight

Parameter containing:
tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519],
        [-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]], requires_grad=True)

## 一个 N-Gram 语言模型

之前我们的 embeds 矩阵的初始值是随机生成的，现在我们将要计算一些训练样例的损失函数，然后使用反向传播去更新参数。

### 文本处理

首先设定参数：

In [10]:
CONTEXT_SIZE = 2  # 指上下文的size
EMBEDDING_DIM = 10

其中，上下文是指目标单词的前 $ n $ 个单词，或者后 $ n $ 个单词，或者前后都有总共 $ n $ 个单词。

在本例中，我们使用的是莎士比亚的十四行诗，使用空格分词：

In [11]:
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

>SONNET 2：
>
>当四十个冬天围攻你的朱颜，
>
>在你美的园地挖下深的战壕，
>
>你青春的华服，那么被人艳羡，
>
>将成褴褛的败絮，谁也不要瞧：
>
>那时人若问起你的美在何处，
>
>哪里是你那少壮年华的宝藏，
>
>你说，"在我这双深陷的眼眶里，
>
>是贪婪的羞耻，和无益的颂扬。"
>
>你的美的用途会更值得赞美，
>
>如果你能够说，"我这宁馨小童
>
>将总结我的账，宽恕我的老迈，"
>
>证实他的美在继承你的血统！
>
>这将使你在衰老的暮年更生，
>
>并使你垂冷的血液感到重温。
>

In [12]:
test_sentence

['When',
 'forty',
 'winters',
 'shall',
 'besiege',
 'thy',
 'brow,',
 'And',
 'dig',
 'deep',
 'trenches',
 'in',
 'thy',
 "beauty's",
 'field,',
 'Thy',
 "youth's",
 'proud',
 'livery',
 'so',
 'gazed',
 'on',
 'now,',
 'Will',
 'be',
 'a',
 "totter'd",
 'weed',
 'of',
 'small',
 'worth',
 'held:',
 'Then',
 'being',
 'asked,',
 'where',
 'all',
 'thy',
 'beauty',
 'lies,',
 'Where',
 'all',
 'the',
 'treasure',
 'of',
 'thy',
 'lusty',
 'days;',
 'To',
 'say,',
 'within',
 'thine',
 'own',
 'deep',
 'sunken',
 'eyes,',
 'Were',
 'an',
 'all-eating',
 'shame,',
 'and',
 'thriftless',
 'praise.',
 'How',
 'much',
 'more',
 'praise',
 "deserv'd",
 'thy',
 "beauty's",
 'use,',
 'If',
 'thou',
 'couldst',
 'answer',
 "'This",
 'fair',
 'child',
 'of',
 'mine',
 'Shall',
 'sum',
 'my',
 'count,',
 'and',
 'make',
 'my',
 'old',
 "excuse,'",
 'Proving',
 'his',
 'beauty',
 'by',
 'succession',
 'thine!',
 'This',
 'were',
 'to',
 'be',
 'new',
 'made',
 'when',
 'thou',
 'art',
 'old,',
 

按道理，我们在训练之前输入的数据应当是有标记的，我们暂时忽略这个事情（后面会说明）。

然后我们要建立一个元组（tuple），元组的格式为：

$$ ([word_{i-2}, word_{i-1}], word_{i}) $$

其中 $word_{i}$ 就是目标词 $ target word $ 

根据以上方法遍历整首诗，然后生成多个元组，放入列表 trigrams 中。

In [13]:
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


set() 函数创建一个无序不重复元素集，可以剔除掉重复的元素，同时可能可以在部分模型如 RNN 中打乱文档上下文，忽略掉 RNN 权重偏向后输入的词语的缺点。

In [14]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

### 搭建网络

接下来我们构建一个网络，名叫 NGramLanguageModeler：

In [15]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

从上述网络来看，init 中存在三个层，分别是：

- 词嵌入层（embedding层）
- 第一个全连接层（linear1）
- 第二个全连接层（linear2）

算上前向传播的过程就是：

- 词嵌入层
- 转换为一维
- 第一个全连接层
- ReLU激活
- 第二个全连接层
- log_softmax输出

### 设置损失函数和优化函数

- 使用NLLLoss()，这个函数其实就是一个不带 log_softmax 的交叉熵损失函数。

- 使用optim.SGD()，最传统的随机梯度下降，没什么好说的。

需要注意的是，model 的输入参数有三个，第一个就是去重后的上下文三词元组数量，第二三个则是我们设定好的 CONTEXT_SIZE 和 EMBEDDING_DIM。

In [16]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

### 训练模型

注意：
```Python
for context, target in trigrams
```
这句话指的是我们的训练样本是上下文（其实只有上文），标签是当前词。这也是为什么之前没有标记的原因。

In [17]:
for epoch in range(100):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[521.0980648994446, 518.3730139732361, 515.6657829284668, 512.9775567054749, 510.3095016479492, 507.6589107513428, 505.0247368812561, 502.40775179862976, 499.8054358959198, 497.2182126045227, 494.64570355415344, 492.0870933532715, 489.54102635383606, 487.00549697875977, 484.47964549064636, 481.962007522583, 479.45440554618835, 476.95621156692505, 474.46732568740845, 471.9869587421417, 469.5133340358734, 467.04665446281433, 464.588490486145, 462.1365964412689, 459.6906900405884, 457.25116419792175, 454.8176681995392, 452.3915226459503, 449.9703652858734, 447.5540897846222, 445.1441671848297, 442.7363283634186, 440.333039522171, 437.93455052375793, 435.5385262966156, 433.1439492702484, 430.75138878822327, 428.3579592704773, 425.9627494812012, 423.56593012809753, 421.16786646842957, 418.7679147720337, 416.3632891178131, 413.9549345970154, 411.538724899292, 409.11780762672424, 406.6911025047302, 404.26082825660706, 401.8231449127197, 399.3772203922272, 396.9234616756439, 394.4617545604706,

现在我们得到了一个损失逐渐降低的模型，可以看出这个模型的结果是收敛的，如果继续训练下去，可能会更准确，但是到此为止吧。

## 练习：计算词嵌入：CBOW模型

连续词袋模型（Continuous Bag-of-Words model ），又被称为CBOW模型，经常用于自然语言处理深度学习。

CBOW模型使用一段文本的中间词作为目标词，即给出目标词的上下文单词作为训练数据。CBOW去除了上下文各词语的词序信息，使用的是上下文各词的词向量的平均值。

CBOW对于一段长度为 $ n $ 的训练样本：$ w_{i-(n-1)}, ...,w_i $ ，其输入为: $$ x = \frac {1}{n - 1}\sum_{w_j \in c}e(w_j) $$

其中 $ w $ 是目标词，即 $ w_i $， $ c $ 是上下文，上式就是将输入求平均。